In [3]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
consumer_key = "YOUR CONSUMER KEY" 
consumer_secret = "YOUR CONSUMER SECRET"
access_token = "YOUR TOKEN"
access_token_secret = "YOUR TOKEN SECRET"

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [4]:
# Target News Account
target_users = ["@BBCNews", "@CBS", "@CNN", "@FoxNews","@nytimes"]

# List to hold Target News sentiments results
results_list = []

# Loop through all target News Accounts on Twitter
for target_user in target_users:
    
    # Variables for holding sentiments for target News Accounts
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Variable for max_id
    oldest_tweet = None

    # Loop through 5 pages of tweets (total 100 tweets)
    for x in range(5):

        # Get all tweets from home feed
        public_tweets = api.user_timeline(target_user, max_id = oldest_tweet, page=x)
        
        # Loop through all tweets 
        for tweet in public_tweets:

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]
            
            # Add each value to the appropriate list
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)

            # Get Tweet ID, subtract 1, and assign to oldest_tweet
            oldest_tweet = tweet['id'] - 1

    
    # Store the Average Sentiments for all target News Accounts
    sentiment = {
        "User": target_user,
        "Compound": np.mean(compound_list),
        "Positive": np.mean(positive_list),
        "Neutral": np.mean(negative_list),
        "Negative": np.mean(neutral_list),
        "Tweet Count": len(compound_list)
    }

    # Print the Sentiments
    print(sentiment)
    print()

    # Append target News results to 'results_list'
    results_list.append(sentiment)

{'User': '@BBCNews', 'Compound': -0.157693, 'Positive': 0.05026, 'Neutral': 0.13734, 'Negative': 0.8123999999999999, 'Tweet Count': 100}

{'User': '@CBS', 'Compound': 0.340953, 'Positive': 0.16154, 'Neutral': 0.02249, 'Negative': 0.8159500000000002, 'Tweet Count': 100}

{'User': '@CNN', 'Compound': -0.051200999999999997, 'Positive': 0.04633, 'Neutral': 0.06387999999999999, 'Negative': 0.8897700000000001, 'Tweet Count': 100}

{'User': '@FoxNews', 'Compound': -0.053066, 'Positive': 0.06896, 'Neutral': 0.09573000000000001, 'Negative': 0.8353100000000001, 'Tweet Count': 100}

{'User': '@nytimes', 'Compound': -0.023621000000000003, 'Positive': 0.04855, 'Neutral': 0.06141, 'Negative': 0.8900100000000001, 'Tweet Count': 100}



In [ ]:
# Convert target News sentiments to DataFrame
sentiments_pd = pd.DataFrame(results_list).set_index("User").round(3)
sentiments_pd.head()